In [1]:
import pandas as pd
import jieba
import jieba.posseg as pseg

In [2]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel

In [3]:
data = pd.read_csv("data.csv")
data.head()

,month,day,title,content
0,2月,27,李开复旗下创新奇智成立新公司,启信宝数据显示，2月26日，李开复担任董事长的深圳创新奇智科技有限公司新增对外投资，成立创新...
1,2月,27,西安市三部门联合印发《西安市规范自动驾驶车辆测试指导意见（试行）》,为优化汽车产业创新发展环境，积极支持并有效规范自动驾驶车辆开展相关测试工作，推动自动驾驶技术...
2,2月,27,联想总裁杨元庆减持联想智慧医疗股份,启信宝数据显示，2月27日，杨元庆持股北京联想智慧医疗信息技术有限公司的比例从1.89%下降...
3,2月,27,中国成为全球最大金融科技投资市场,埃森哲最新研究显示，2018年中国成为全球最大的金融科技投资市场。去年，全球金融科技投资总额...
4,2月,27,拳打Adam，脚踢SGD：北大提出全新优化算法AdaBound,在论文《Adaptive Gradient Methods with Dynamic Bou...


In [4]:
stop_list = ['有限公司', '控股公司', '街道', '部门', '指南', '外媒',
              '财年', '县', '技术', '成本', '消息', '热议', 'com',
             '纲要', '基础', '计划', '功能', '网关', '意图', '项目',
             '发货', '用户', '订单', '省', '高地', '协议', '部', '愿景',
             '贡献', '结果', '方法', '行动计划', '内', '资金', 'Ctrl',
             '短语', '研究者', '产生', '问题', '太若', '经济', '农机', 
             '餐厅', '一体', '技术', '结果', '利用', '参数', '领域',
             'D', 'G', '情况', '病', '目的', '装备', '作者', '数据',
            'Xiong', 'Yan', 'Sun', 'Liu', 'Yuanhao', 'Xu', '能力', 
            '体系', '战略', '效能', '时代', '总书记', '手段', '有力', '意见',
            '战略', '行业', '博士', '研究生', '致力于', '充分利用', '感兴趣', '部分',
            '企业', '法定代表', '注册资本', '工作人员', '外界', '面积', '人员',
            '方式', '全面', '机构', '部署', '工程', '大力支持', '人才', '公司', '方面']

In [5]:
added_words = ['创新奇智', '创新工场', '自动驾驶', '无人驾驶', '阿尔兹海默']
for i in added_words:
    jieba.add_word(i)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/py/_2k08w6n22164dfm66tnx57w0000gn/T/jieba.cache
Loading model cost 0.848 seconds.
Prefix dict has been built succesfully.


In [6]:
common_texts = []
for news in data.content:
    seg_list = []
    words = pseg.cut(news)
    for w in words:
        #   保留名词词汇和英文
        if (w.flag in ["n", "nr", "nz", "nt", "nw", "eng"] and w.word not in stop_list and len(w.word)>1) or (w.word in added_words):
#         if w.flag in ["n", "nr", "nz", "nt", "nw", "eng"]:
            seg_list.append(w.word)
    common_texts.append(list(seg_list))

In [7]:
# Create a corpus from a list of texts
common_dictionary = Dictionary(common_texts)
common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]

In [8]:
print(len(common_dictionary))
print(len(common_corpus))
# common_dictionary.token2id

2559
279


In [9]:
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=50, minimum_probability=0.02)

In [21]:
# 热门词统计
hottest_words = {}

In [22]:
# mm = 0
# nn = 0
topic_words = []

for doc in common_corpus:
#     mm += 1
    cutted_words = []
    for i in doc:
        cutted_words.append(common_dictionary[i[0]])
    for topic in (lda.get_document_topics(doc, minimum_probability=0.5)):
#         nn += 1
        words = lda.show_topic(topic[0], topn=100)
        s = []
        for w in words:
            if common_dictionary[int(w[0])] in cutted_words:
                s.append((common_dictionary[int(w[0])], round(w[1], 4)))
                if common_dictionary[int(w[0])] in hottest_words.keys():
                    hottest_words[common_dictionary[int(w[0])]] += 1
                else:
                    hottest_words[common_dictionary[int(w[0])]] = 1
        topic_words.append(s)
        
# print(mm, nn)

In [23]:
print(len(topic_words))
# print(hottest_words)

279


In [24]:
# 文章和主题词
data["topic_words"] = topic_words
data.sort_values(by="day", inplace=True)
data.to_csv("NewsAndTopics.csv", index=False)

In [28]:
# 一周热门词
hwDF = pd.DataFrame(columns=['topic_words', 'score(appearances)'])
hwDF['topic_words'] = hottest_words.keys()
hwDF['score(appearances)'] = hottest_words.values()

In [31]:
hwDF.sort_values(by="score(appearances)", ascending=False, inplace=True)
hwDF.to_csv("TopicWordsSorted.csv", index=False)

In [32]:
hwDF

,topic_words,score(appearances)
69,人工智能,83
82,智能,66
3,AI,48
0,科技,47
338,系统,42
85,产品,42
29,全球,39
90,平台,35
151,机器人,34
161,机器,32
